In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv('../input/kidney-disease-data/kidney_disease.csv')
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [4]:
# drop id col
df = df.iloc[:,1:]

# # Separate object type datatype
df_object_feat = df.select_dtypes(include='object')
df_object_feat_name = df_object_feat.columns.values
df_object_feat_name

array(['rbc', 'pc', 'pcc', 'ba', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'], dtype=object)

In [5]:
# import encoder
from sklearn.preprocessing import OrdinalEncoder
ode = OrdinalEncoder(dtype=int)


In [6]:
# # retain non -null values
# nonulls = np.array(df['rbc'].dropna())

# # convert into array with one column
# nonulls_reshape= nonulls.reshape(-1,1)

# #encode the data
# impute_ordinal = ode.fit_transform(nonulls_reshape)

# # assign back encoded values to non null values
# df['rbc'].loc[df['rbc'].notnull()] = np.squeeze(impute_ordinal)
# df['rbc']

In [7]:
# define encodeing fun
def encode(data):
    # retain non -null values
    nonulls = np.array(data.dropna())

    # convert into array with one column
    nonulls_reshape= nonulls.reshape(-1,1)

    #encode the data
    impute_ordinal = ode.fit_transform(nonulls_reshape)

    # assign back encoded values to non null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

In [8]:
# call the function
for feat in df_object_feat_name:
    encode(df[feat])

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [9]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,1,0,0,121.0,...,30,69,30,1,1,0,0,0,0,0
1,7.0,50.0,1.020,4.0,0.0,NaN,1,0,0,NaN,...,24,53,NaN,0,0,0,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,1,1,0,0,423.0,...,17,67,NaN,0,1,0,1,0,1,0
3,48.0,70.0,1.005,4.0,0.0,1,0,1,0,117.0,...,18,59,17,1,0,0,1,1,1,0
4,51.0,80.0,1.010,2.0,0.0,1,1,0,0,106.0,...,21,65,24,0,0,0,0,0,0,0


In [10]:
#KNN imputer
knn  = KNNImputer(n_neighbors=4)
cleandata = knn.fit_transform(df)
#Convert to Data FrameObject
cleandata = pd.DataFrame(cleandata,columns=df.columns)
cleandata.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,0.75,1.0,0.0,0.0,121.0,...,30.0,69.00,30.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,7.0,50.0,1.020,4.0,0.0,0.50,1.0,0.0,0.0,169.5,...,24.0,53.00,25.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62.0,80.0,1.010,2.0,3.0,1.00,1.0,0.0,0.0,423.0,...,17.0,67.00,24.75,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,48.0,70.0,1.005,4.0,0.0,1.00,0.0,1.0,0.0,117.0,...,18.0,59.00,17.00,1.0,0.0,0.0,1.0,1.0,1.0,0.0
4,51.0,80.0,1.010,2.0,0.0,1.00,1.0,0.0,0.0,106.0,...,21.0,65.00,24.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,60.0,90.0,1.015,3.0,0.0,0.50,0.5,0.0,0.0,74.0,...,25.0,69.00,22.00,1.0,1.0,0.0,0.0,1.0,0.0,0.0
6,68.0,70.0,1.010,0.0,0.0,0.75,1.0,0.0,0.0,100.0,...,22.0,40.25,23.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,24.0,70.0,1.015,2.0,4.0,1.00,0.0,0.0,0.0,410.0,...,30.0,61.00,28.00,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,52.0,100.0,1.015,3.0,0.0,1.00,0.0,1.0,0.0,138.0,...,19.0,85.00,18.00,1.0,1.0,0.0,0.0,0.0,1.0,0.0
9,53.0,90.0,1.020,2.0,0.0,0.00,0.0,1.0,0.0,70.0,...,15.0,15.00,15.00,1.0,1.0,0.0,1.0,0.0,1.0,0.0
